In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

education_df = pd.read_sql_query('select * from useducation',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

In [3]:
# We fill in the missing values using interpolation since our data is a time series data
fill_list = ["ENROLL", "TOTAL_REVENUE", "FEDERAL_REVENUE",
             "STATE_REVENUE", "LOCAL_REVENUE", "TOTAL_EXPENDITURE",
             "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
             "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE", "GRADES_PK_G",
             "GRADES_KG_G", "GRADES_4_G", "GRADES_8_G", "GRADES_12_G", "GRADES_1_8_G",
             "GRADES_9_12_G", "GRADES_ALL_G"]

states = education_df["STATE"].unique()

for state in states:
    education_df.loc[education_df["STATE"] == state, fill_list] = education_df.loc[education_df["STATE"] == state, fill_list].interpolate()

# we drop the null values after interpolation
education_df.dropna(inplace=True)

In [4]:
education_df.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
209,1996_ALABAMA,ALABAMA,1996,735912.0,3365113.0,334858.0,2138717.0,891538.0,3572283.0,1987018.0,...,57098.0,58305.0,42510.0,469370.0,207980.0,686983.0,211.646974,256.594863,210.920767,260.276441
211,1996_ARIZONA,ARIZONA,1996,764681.0,4143304.0,378917.0,1798363.0,1966024.0,4391555.0,1860007.0,...,62793.0,60514.0,42041.0,511925.0,210289.0,726709.0,217.575940,267.874834,206.529788,263.283638
212,1996_ARKANSAS,ARKANSAS,1996,452907.0,2183384.0,192152.0,1296247.0,694985.0,2297381.0,1264275.0,...,33855.0,37064.0,27613.0,283883.0,131997.0,417860.0,215.846436,261.652745,208.837716,271.834590
213,1996_CALIFORNIA,CALIFORNIA,1996,5460484.0,31282981.0,2603882.0,17061474.0,11617625.0,31561692.0,16880730.0,...,435217.0,406456.0,298669.0,3516572.0,1525618.0,5115423.0,209.129083,262.772478,201.912591,269.197451
214,1996_COLORADO,COLORADO,1996,655679.0,3891203.0,194998.0,1664676.0,2031529.0,4075165.0,2052264.0,...,52524.0,52269.0,37179.0,423510.0,185700.0,621730.0,225.805858,275.608214,221.653243,268.168154


1. Create a new score variable from the weighted averages of the all score variables in the datasets. Notice that the number of students in the fourth grade isn't the same with the number of students in the eighth grade. So, you should weight the scores appropriately!.

In [11]:
education_df["overall_score"] = (education_df["GRADES_4_G"]*((education_df["AVG_MATH_4_SCORE"] + education_df["AVG_READING_4_SCORE"])*0.5) + education_df["GRADES_8_G"]
                                 * ((education_df["AVG_MATH_8_SCORE"] + education_df["AVG_READING_8_SCORE"])*0.5))/(education_df["GRADES_4_G"]+education_df['GRADES_8_G']+
education_df['GRADES_12_G']+education_df['GRADES_1_8_G']+education_df['GRADES_9_12_G']+education_df['GRADES_ALL_G'])

In [12]:
education_df["overall_score"]

209     17.823648
211     18.204315
212     18.315930
213     17.486116
214     18.910531
          ...    
1487    18.927503
1488    18.330893
1489    18.019487
1490    17.542094
1491    19.166298
Name: overall_score, Length: 466, dtype: float64

2. What are the correlations between this newly created score variable and the expenditure types? Which one of the expenditure type is more correlated than the others?

In [13]:
education_df[["overall_score", "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE",
              "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]]

,overall_score,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
209,17.823648,3572283.0,1987018.0,935115.0,322883.0,286699.0
211,18.204315,4391555.0,1860007.0,1193260.0,220650.0,877190.0
212,18.315930,2297381.0,1264275.0,636106.0,127007.0,203555.0
213,17.486116,31561692.0,16880730.0,9222569.0,2362407.0,2580639.0
214,18.910531,4075165.0,2052264.0,1127563.0,143767.0,588209.0
...,...,...,...,...,...,...
1487,18.927503,16113212.0,8755896.0,5075509.0,627473.0,1086722.0
1488,18.330893,13630138.0,6508964.0,4510672.0,546926.0,1601069.0
1489,18.019487,3466981.0,1819903.0,1161944.0,233836.0,232738.0
1490,17.542094,11553677.0,5723474.0,3691809.0,513402.0,894823.0


In [16]:
# Now, apply PCA to the four expenditure types. How much of the total variance is explained by the first component?

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = education_df[["INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
                  "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]]

X = StandardScaler().fit_transform(X)

sklearn_pca = PCA(n_components=1)
education_df["pca_1"] = sklearn_pca.fit_transform(X)

print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    sklearn_pca.explained_variance_ratio_
)

The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 [0.9430175]


In [17]:
# More than 94% of the total variance is explained by the first principal component.

What is the correlation between the overall score variable and the first principal component?

In [20]:
education_df[["overall_score", "pca_1", "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE",
              "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]]

,overall_score,pca_1,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
209,17.823648,-0.939455,3572283.0,1987018.0,935115.0,322883.0,286699.0
211,18.204315,-0.802867,4391555.0,1860007.0,1193260.0,220650.0,877190.0
212,18.315930,-1.219663,2297381.0,1264275.0,636106.0,127007.0,203555.0
213,17.486116,3.646088,31561692.0,16880730.0,9222569.0,2362407.0,2580639.0
214,18.910531,-0.958451,4075165.0,2052264.0,1127563.0,143767.0,588209.0
...,...,...,...,...,...,...,...
1487,18.927503,0.595097,16113212.0,8755896.0,5075509.0,627473.0,1086722.0
1488,18.330893,0.469425,13630138.0,6508964.0,4510672.0,546926.0,1601069.0
1489,18.019487,-1.012990,3466981.0,1819903.0,1161944.0,233836.0,232738.0
1490,17.542094,0.044108,11553677.0,5723474.0,3691809.0,513402.0,894823.0


If you were to choose the best variables for your model, would you prefer using the first principal component instead of the expenditure variables? Why?

Instruction expenditure variable is more correlated with the overall score than the first principal component. Hence using instruction expenditure makes more sense. PCA works best when the correlation between the variables are less than and equal to 0.8. In our case, all of the expenditure variables are highly correlated with each other. This may result in instable principal component estimations.